**Updating the System Environment**

In [7]:
!sudo apt update

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
42 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InR

**Installing JDK**

In [8]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

**Downloading Apache Spark Distribution**

In [10]:

!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

**Extracting the Downloaded Apache Spark File**

In [11]:
!tar xf /content/spark-3.4.3-bin-hadoop3.tgz

**Installing Spark**

In [12]:
!pip install -q findspark

In [ ]:
!pip install pyspark

In [13]:
!pip install py4j

**Importing Necessary Libraries**

In [14]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T




**Starting Pyspark Session**

In [15]:
spark = SparkSession \
        .builder \
        .appName("PySpark Shell") \
        .getOrCreate()

spark

We Are Using The US_youtube_trending_data.
The Dataset is from Kaggle.
Dataset Link is: https://www.kaggle.com/datasets/rsrishav/youtube-trending-video-dataset/data?select=US_youtube_trending_data.csv

The Dataset Size is around 390.2 MB. It;s been stored in Google Drive.

Now we will mount google drive here.

**Mounting Google Drive**

In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Setting Up the Dataset Path**

In [18]:
path_dataset = "/content/drive/MyDrive/Big Data Processing/US_youtube_trending_data.csv"

**Importing and Reading the Data**

In [19]:
df = spark.read.option("wholeFile",True) \
               .option("multiline",True) \
               .option("header", True) \
               .option("inferSchema", True) \
               .csv(path_dataset)

**Basic Exploaration and Row Count of The Dataset**

In [20]:
df.describe().show()

+-------+-----------+--------------------+--------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+
|summary|   video_id|               title|           channelId|        channelTitle|        categoryId|                tags|        view_count|             likes|         dislikes|     comment_count|      thumbnail_link|         description|
+-------+-----------+--------------------+--------------------+--------------------+------------------+--------------------+------------------+------------------+-----------------+------------------+--------------------+--------------------+
|  count|     268787|              268787|              268787|              268787|            268787|              268787|            268787|            268787|           268787|            268787|              268787|              264238|
|   mean|       NULL|  31.657142

In [21]:
number_of_rows = df.count()

In [22]:
number_of_rows

268787

In [23]:
df.show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|      trending_date|                tags|view_count| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|3C66w5Z0ixs|I ASKED HER TO BE...|2020-08-11 19:20:14|UCvtRTOMP2TqYqu51...|            Brawadis|        22|2020-08-12 00:00:00|brawadis|prank|ba...|   1514614|156908|    5855|        35313|https://i.

**Schema Changes to Two Time-Related Variables**

In [24]:
from pyspark.sql.functions import to_timestamp
# Alter the data type of the "publishedAt" column to timestamp
df = df.withColumn("publishedAt", to_timestamp(df["publishedAt"], "yyyy-MM-dd'T'HH:mm:ss'Z'"))

# Show the DataFrame with the altered data type
df.printSchema()

root
 |-- video_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- publishedAt: timestamp (nullable = true)
 |-- channelId: string (nullable = true)
 |-- channelTitle: string (nullable = true)
 |-- categoryId: integer (nullable = true)
 |-- trending_date: timestamp (nullable = true)
 |-- tags: string (nullable = true)
 |-- view_count: integer (nullable = true)
 |-- likes: integer (nullable = true)
 |-- dislikes: integer (nullable = true)
 |-- comment_count: integer (nullable = true)
 |-- thumbnail_link: string (nullable = true)
 |-- comments_disabled: boolean (nullable = true)
 |-- ratings_disabled: boolean (nullable = true)
 |-- description: string (nullable = true)



In [25]:
df.show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|      trending_date|                tags|view_count| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|         description|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+--------------------+
|3C66w5Z0ixs|I ASKED HER TO BE...|2020-08-11 19:20:14|UCvtRTOMP2TqYqu51...|            Brawadis|        22|2020-08-12 00:00:00|brawadis|prank|ba...|   1514614|156908|    5855|        35313|https://i.

**Check for Missing Values**

In [26]:
from pyspark.sql.functions import col

# Iterate over each column and check for missing values
columns_with_missing_values = []

for column in df.columns:
    missing_count = df.where(col(column).isNull()).count()
    if missing_count > 0:
        columns_with_missing_values.append(column)

# Print the columns with missing values
print("Columns with missing values:", columns_with_missing_values)

Columns with missing values: ['description']


Description column has missing values, more clearly empty values. We will drop description column from the dataframe.

**Dropping the description Column**

In [27]:
# Drop the "description" column
df = df.drop("description")

# Show the DataFrame after dropping the column
df.show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|      trending_date|                tags|view_count| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+
|3C66w5Z0ixs|I ASKED HER TO BE...|2020-08-11 19:20:14|UCvtRTOMP2TqYqu51...|            Brawadis|        22|2020-08-12 00:00:00|brawadis|prank|ba...|   1514614|156908|    5855|        35313|https://i.ytimg.c...|            false|           false|
|M9Pmf9AB4Mo|Ape

**Checking for Duplicate Data**

In [28]:
# Check for duplicate rows
duplicate_rows = df.groupBy(df.columns).count().filter("count > 1")

# Count the number of duplicate rows
num_duplicate_rows = duplicate_rows.count()

# Show the count of duplicate rows
print("Number of duplicate rows:", num_duplicate_rows)

# Show duplicate rows
if num_duplicate_rows > 0:
    duplicate_rows.show()

Number of duplicate rows: 83
+-----------+-----------------------+-------------------+--------------------+--------------------+----------+-------------------+---------------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+-----+
|   video_id|                  title|        publishedAt|           channelId|        channelTitle|categoryId|      trending_date|                       tags|view_count| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|count|
+-----------+-----------------------+-------------------+--------------------+--------------------+----------+-------------------+---------------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+-----+
|beJD7y1mGLA|   Awkwafina Gets Ho...|2021-02-18 16:00:21|UCPD_bxCRGpmmeQcb...|      First We Feast|        24|2021-02-24 00:00:00|       fwf|first we feas...|    808492| 26693|     709|

Here is total number of 83 duplicate records, we will remove these 83 duplicate records.

**Removing the Duplicates**

In [29]:
# Remove duplicate rows
df_no_duplicates = df.dropDuplicates()

# Show the DataFrame without duplicates
df_no_duplicates.show()
df_no_duplicates.count()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|      trending_date|                tags|view_count| likes|dislikes|comment_count|      thumbnail_link|comments_disabled|ratings_disabled|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+--------------------+-----------------+----------------+
|gPdUslndvVI|Our Farm Got Dest...|2020-08-11 23:00:06|UCuxlXCfVyV-i5YLL...|   Cole The Cornstar|        22|2020-08-12 00:00:00|farming|family fa...|    277338| 37533|     197|         3666|https://i.ytimg.c...|            false|           false|
|Que4RnxYLNc|ian

268704

**Dropping thumbnail link columns**

In [30]:
df_no_duplicates = df_no_duplicates.drop("thumbnail_link")



In [31]:
df_no_duplicates.show()

+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+-----------------+----------------+
|   video_id|               title|        publishedAt|           channelId|        channelTitle|categoryId|      trending_date|                tags|view_count| likes|dislikes|comment_count|comments_disabled|ratings_disabled|
+-----------+--------------------+-------------------+--------------------+--------------------+----------+-------------------+--------------------+----------+------+--------+-------------+-----------------+----------------+
|gPdUslndvVI|Our Farm Got Dest...|2020-08-11 23:00:06|UCuxlXCfVyV-i5YLL...|   Cole The Cornstar|        22|2020-08-12 00:00:00|farming|family fa...|    277338| 37533|     197|         3666|            false|           false|
|Que4RnxYLNc|iann dior – Prosp...|2020-08-11 19:15:10|UCtpzjafUx5qbmxJd...|           iann dior|    

**Importing Category Id Related Json Data**

In [32]:
import json
cat = open('/content/drive/MyDrive/Big Data Processing/US_category_id.json')
catid = json.load(cat)

In [33]:
catid

{'kind': 'youtube#videoCategoryListResponse',
 'etag': 'HIrK3n45Uw2IYz9_U2-gK1OsXvo',
 'items': [{'kind': 'youtube#videoCategory',
   'etag': 'IfWa37JGcqZs-jZeAyFGkbeh6bc',
   'id': '1',
   'snippet': {'title': 'Film & Animation',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': '5XGylIs7zkjHh5940dsT5862m1Y',
   'id': '2',
   'snippet': {'title': 'Autos & Vehicles',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': 'HCjFMARbBeWjpm6PDfReCOMOZGA',
   'id': '10',
   'snippet': {'title': 'Music',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': 'ra8H7xyAfmE2FewsDabE3TUSq10',
   'id': '15',
   'snippet': {'title': 'Pets & Animals',
    'assignable': True,
    'channelId': 'UCBR8-60-B28hp2BmDPdntcQ'}},
  {'kind': 'youtube#videoCategory',
   'etag': '7mqChSJogdF3hSIL-88BfDE-W8M',
   'id': '

**Data Extraction from the Json**

In [34]:
catid_sim = []
for item in catid["items"]:
    catid_sim.append({"id": item["id"], "title": item["snippet"]["title"]})

# Display the simplified list
for category in catid_sim:
    print(category)

{'id': '1', 'title': 'Film & Animation'}
{'id': '2', 'title': 'Autos & Vehicles'}
{'id': '10', 'title': 'Music'}
{'id': '15', 'title': 'Pets & Animals'}
{'id': '17', 'title': 'Sports'}
{'id': '18', 'title': 'Short Movies'}
{'id': '19', 'title': 'Travel & Events'}
{'id': '20', 'title': 'Gaming'}
{'id': '21', 'title': 'Videoblogging'}
{'id': '22', 'title': 'People & Blogs'}
{'id': '23', 'title': 'Comedy'}
{'id': '24', 'title': 'Entertainment'}
{'id': '25', 'title': 'News & Politics'}
{'id': '26', 'title': 'Howto & Style'}
{'id': '27', 'title': 'Education'}
{'id': '28', 'title': 'Science & Technology'}
{'id': '29', 'title': 'Nonprofits & Activism'}
{'id': '30', 'title': 'Movies'}
{'id': '31', 'title': 'Anime/Animation'}
{'id': '32', 'title': 'Action/Adventure'}
{'id': '33', 'title': 'Classics'}
{'id': '34', 'title': 'Comedy'}
{'id': '35', 'title': 'Documentary'}
{'id': '36', 'title': 'Drama'}
{'id': '37', 'title': 'Family'}
{'id': '38', 'title': 'Foreign'}
{'id': '39', 'title': 'Horror'}


In [35]:
import pyspark

schema = pyspark.sql.types.StructType([
    pyspark.sql.types.StructField("id", pyspark.sql.types.StringType(), nullable=False),
    pyspark.sql.types.StructField("title", pyspark.sql.types.StringType(), nullable=False)
])

# Create a DataFrame from the catid_sim list using the defined schema
catdf = spark.createDataFrame(catid_sim, schema=schema)

# Show the DataFrame
catdf.show()

+---+--------------------+
| id|               title|
+---+--------------------+
|  1|    Film & Animation|
|  2|    Autos & Vehicles|
| 10|               Music|
| 15|      Pets & Animals|
| 17|              Sports|
| 18|        Short Movies|
| 19|     Travel & Events|
| 20|              Gaming|
| 21|       Videoblogging|
| 22|      People & Blogs|
| 23|              Comedy|
| 24|       Entertainment|
| 25|     News & Politics|
| 26|       Howto & Style|
| 27|           Education|
| 28|Science & Technology|
| 29|Nonprofits & Acti...|
| 30|              Movies|
| 31|     Anime/Animation|
| 32|    Action/Adventure|
+---+--------------------+
only showing top 20 rows



In [36]:
catdf = catdf.withColumnRenamed("title", "category")

# Show the DataFrame with the renamed column
catdf.show()

+---+--------------------+
| id|            category|
+---+--------------------+
|  1|    Film & Animation|
|  2|    Autos & Vehicles|
| 10|               Music|
| 15|      Pets & Animals|
| 17|              Sports|
| 18|        Short Movies|
| 19|     Travel & Events|
| 20|              Gaming|
| 21|       Videoblogging|
| 22|      People & Blogs|
| 23|              Comedy|
| 24|       Entertainment|
| 25|     News & Politics|
| 26|       Howto & Style|
| 27|           Education|
| 28|Science & Technology|
| 29|Nonprofits & Acti...|
| 30|              Movies|
| 31|     Anime/Animation|
| 32|    Action/Adventure|
+---+--------------------+
only showing top 20 rows



**Merging Both Json and Loaded Dataset**

In [37]:

# Merging both dataset according to category ID
from pyspark.sql.functions import col
catdf = catdf.withColumn("id", catdf.id.cast('int'))
df_no_duplicates = df_no_duplicates.join(catdf,df_no_duplicates.categoryId == catdf.id,"inner")

In [ ]:
df_no_duplicates.show(10)

+-----------+--------------------+-------------------+--------------------+-------------------+----------+-------------------+--------------------+----------+------+--------+-------------+-----------------+----------------+---+----------------+
|   video_id|               title|        publishedAt|           channelId|       channelTitle|categoryId|      trending_date|                tags|view_count| likes|dislikes|comment_count|comments_disabled|ratings_disabled| id|        category|
+-----------+--------------------+-------------------+--------------------+-------------------+----------+-------------------+--------------------+----------+------+--------+-------------+-----------------+----------------+---+----------------+
|gSfq4Wbr264|DanPlan Animated ...|2020-09-09 19:47:18|UC7UGbBVrqLlq6CRx...|            DanPlan|         1|2020-09-14 00:00:00|dan plan|by the w...|    535785| 55458|    1792|         7031|            false|           false|  1|Film & Animation|
|3MRQxyZZYcM|TRYING 

In [38]:
df_no_duplicates.count()

268704

**Exporting the Processed Dataset into One Single CSV File**

In [39]:
df_no_duplicates.repartition(1).write.option("header",True).csv("/content/drive/MyDrive/Big Data Processing/US_YouTube_Trending_Final_Merged_Processed2.csv")